# Building the simplest Graph

We start with a graph with two nodes connected by one edge. 


In [1]:
!pip install langgraph

Nodes act like functions that can be called as needed. In our case Node 1 is our starting point and Node 2 is our finish point.

In [1]:
def function_1(input_1):
    return input_1 + " Hi "

def function_2(input_2):
    return input_2 + "there"

In [2]:
from langgraph.graph import Graph

# Define a Langchain graph
workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('node_1', 'node_2')

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [3]:
app.invoke("Hello")

'Hello Hi there'

In [6]:
input = 'Hello'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output '{value}'from node '{key}':")
    print("\n---\n")

Output 'Hello Hi 'from node 'node_1':

---

Output 'Hello Hi there'from node 'node_2':

---



### As you can see, we can run the nodes as functions and return some values from them. 



# Adding LLM Call

Now, let's make the first node as an "Agent" that can call Open AI models. We can use langchain to make this call easy for us. 

In [7]:
!pip install langchain langchain_openai

A usual call to ChatOpenAI model in LangChain is done as below:

First set your API keys for OpenAI

In [8]:
!pip install python-dotenv

In [89]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_BASE'] = os.getenv("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE= os.environ.get("OPENAI_API_BASE")
# Now you can access your environment variables using os.environ
print(OPENAI_API_KEY,OPENAI_API_BASE)

sk-rKcLAkdmwzWdMdOTdlXFKZuLEcY8drtZJ2DsXZZkTC9sQoE0 https://xiaoai.plus/v1


In [9]:
from langchain_openai import ChatOpenAI

# Set the model as ChatOpenAI
model = ChatOpenAI(
            openai_api_base=OPENAI_API_BASE,
            openai_api_key=OPENAI_API_KEY,
            temperature=0)

#Call the model with a user message
model.invoke('Hey there')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'stop', 'logprobs': None}, id='run-8cdba97d-6e05-4a4a-8483-1f627d0fc9fb-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {}, 'output_token_details': {}})

And if you just want to see the AI response, you can do the following:

In [10]:
model.invoke('Hey there').content

'Hello! How can I assist you today?'

Cool! Keeping that in mind, let's change the function 1 above so that we can send the user question to the model. Then we will send this response to function 2, which will add a short string and return to the user.

In [11]:
def function_1(input_1):
    response = model.invoke(input_1)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [12]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [13]:
app.invoke("Hey there")

'Agent Says: Hello! How can I assist you today?'

In [14]:
input = 'Hey there'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output '{value}'is from node '{key}':")
        print("\n---\n")

Output 'Hello! How can I assist you today?'is from node 'agent':

---

Output 'Agent Says: Hello! How can I assist you today?'is from node 'node_2':

---



# First functional Agent App - City Temperature

### Step 1: Parse the city mentioned
Let's extract the city that a user mentions in a query

In [15]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [16]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [17]:
app.invoke("What's the temperature in Las Vegas")

'Agent Says: Las Vegas'

### Step 2: Adding a weather API call

What if we want the function 2 to take the city name and give us the weather for that city.

Well we know that Open Weather Map is [integrated](https://python.langchain.com/docs/integrations/tools/openweathermap) into LangChain

We need to install pyown, create an API key on the website of Open Weather Map (which takes a few hours to activate) and then run the cells below to get weather of a given city.

In [19]:
!pip install pyowm

In [18]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
load_dotenv()
os.environ["OPENWEATHERMAP_API_KEY"] = os.environ.get("OPENWEATHERMAP_API_KEY")

weather = OpenWeatherMapAPIWrapper()

In [20]:
weather_data = weather.run("Las Vegas")
print(weather_data)

In Las Vegas, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 5.14 m/s, direction: 310°
Humidity: 30%
Temperature: 
  - Current: 14.7°C
  - High: 15.69°C
  - Low: 13.71°C
  - Feels like: 13.01°C
Rain: {}
Heat index: None
Cloud cover: 75%


Now, let's integrate this into function 2 and call the function two as a "tool" or "weather_agent" instead of "node_2" in our workflow.

In [21]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    weather_data = weather.run(input_2)
    return weather_data

In [22]:
from langgraph.graph import Graph

workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('agent', 'tool')

workflow.set_entry_point("agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [23]:
app.invoke("What's the temperature in Las Vegas")

'In Las Vegas, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.14 m/s, direction: 310°\nHumidity: 30%\nTemperature: \n  - Current: 14.7°C\n  - High: 15.69°C\n  - Low: 13.71°C\n  - Feels like: 13.01°C\nRain: {}\nHeat index: None\nCloud cover: 75%'

In [24]:
input = "What's the temperature in Las Vegas"
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output '{value}'is from node '{key}':")
        print("\n---\n")

Output 'Las Vegas'is from node 'agent':

---

Output 'In Las Vegas, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 5.14 m/s, direction: 310°
Humidity: 30%
Temperature: 
  - Current: 14.7°C
  - High: 15.69°C
  - Low: 13.71°C
  - Feels like: 13.01°C
Rain: {}
Heat index: None
Cloud cover: 75%'is from node 'tool':

---



### Step 3 Adding another LLM Call to filter results

What if we only want the temperature? But current setup gives us the full weather report. 

Well we can make another LLM call to filter data

In [26]:
def function_3(input_3):
    complete_query = "Your task is to provide info concisely based on the user query. Following is the user query: " + "user input"
    response = model.invoke(complete_query)
    return response.content

But the issue is the user input is not available from node 2.

Can we pass user input all along from first node to the last?

Yes, we can use a dictionary and pass it between nodes (we could also use just a list, but dict makes it a bit easier)

In [25]:
# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []

In [26]:
AgentState

{'messages': []}

Our goal is to have this state filled as:
{'messages': [HumanMessage, AIMessage, ...]]}

Also now we need to modify our functions to pass info along the new AgentState

In [29]:
def function_1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = "Your task is to provide only the city name based on the user query. \
                    Nothing more, just the city name mentioned. Following is the user query: " + user_input
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_2(state):
    messages = state['messages']
    agent_response = messages[-1]
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(agent_response)
    state['messages'].append(weather_data)
    return state

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    agent2_query = "Your task is to provide info concisely based on the user query and the available information from the internet. \
                        Following is the user query: " + user_input + " Available information: " + available_info
    response = model.invoke(agent2_query)
    return response.content


In [30]:
from langgraph.graph import Graph

workflow = Graph()


workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)
workflow.add_node("responder", function_3)

workflow.add_edge('agent', 'tool')
workflow.add_edge('tool', 'responder')

workflow.set_entry_point("agent")
workflow.set_finish_point("responder")

app = workflow.compile()

In [31]:
inputs = {"messages": ["what is the temperature in las vegas"]}
app.invoke(inputs)

'The current temperature in Las Vegas is 14.7°C with broken clouds and a 75% cloud cover.'

In [32]:
input = {"messages": ["what is the temperature in las vegas"]}
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output '{value}'is from node '{key}':")
        print("\n---\n")

Output '{'messages': ['what is the temperature in las vegas', 'Las Vegas']}'is from node 'agent':

---

Output '{'messages': ['what is the temperature in las vegas', 'Las Vegas', 'In Las Vegas, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.14 m/s, direction: 310°\nHumidity: 30%\nTemperature: \n  - Current: 14.7°C\n  - High: 15.69°C\n  - Low: 13.71°C\n  - Feels like: 13.01°C\nRain: {}\nHeat index: None\nCloud cover: 75%']}'is from node 'tool':

---

Output 'The current temperature in Las Vegas is 14.7°C with broken clouds and a 75% cloud cover.'is from node 'responder':

---



As we notice that there is a lot of appending to the array going on, we can make it a bit easier with the following:

```bash
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


It basically makes the state dictionary as saw previously, and also makes sure that any new message is appended to the messages array when we do the following: 
```bash
{"messages": [new_array_element]}
```


##### We also realize that our app is not capable of answering simple questions like "how are you?"

In [33]:
inputs = {"messages": ["how are you?"]}
app.invoke(inputs)

"I am an AI assistant, so I don't have feelings, but thank you for asking. In Istanbul, the current weather is broken clouds with a temperature of 2.35°C, feeling like -2.77°C. The wind speed is 6.69 m/s with a direction of 40° and humidity is at 75%."

This is because we always want to parse a city and then find the weather. 

We can make our agent smarter by saying only use the tool when needed, if not just respond back to the user. 

The way we can do this LangGraph is:
1. binding a tool to the agent
2. adding a conditional edge to the agent with the option to either call the tool or not
3. defining the criteria for the conditional edge as when to call the tool. We will define a function for this.


Let's start with the AgentState definition as mentioned a few cells above.

In [48]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

Binding tool with agent (LLM Model) is made easy in langchain

In [49]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(
            openai_api_base=OPENAI_API_BASE,
            openai_api_key=OPENAI_API_KEY,
            temperature=0)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_tools(functions)

Our modified function_1 now becomes as below. The reason is, we are passing the human message as state and appending response to the state. Also, our agent now has a tool bound to it, that it can use.

In [124]:
def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

For function 2, we want it to setup a tool and call it. It's made easy to invoke a tool in LangChain by using ToolInvocation and executing it with ToolNode . Then we respond back as a FunctionMessage so that our agent (node 1) knows that the tool was used and a response from tool is available.

In [165]:
from langgraph.prebuilt import ToolInvocation, ToolExecutor  # 引入 ToolNode
import json
# from langchain_core.messages import FunctionMessage
from langchain_core.messages import HumanMessage, ToolMessage

tool_executor = ToolExecutor(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1]  # 取最后一条消息，获取要发送给工具的查询

    # 确保 tool_calls 存在且为非空列表
    tool_calls = last_message.additional_kwargs.get("tool_calls", [])

    if tool_calls:
        function_data = tool_calls[0].get("function", {})  # 获取 function 字典
        arguments_str = function_data.get("arguments", "{}")  # 获取 arguments JSON 字符串
        parsed_tool_input = json.loads(arguments_str)  # 解析 JSON
        tool_call_id=tool_calls[0]["id"]
    else:
        print("Warning: tool_calls is empty.")
    
    # 构造 ToolInvocation
    action = ToolInvocation(
        tool=function_data["name"],
        tool_input=parsed_tool_input['location'],
    )

    # 使用 tool_executor 处理请求
    response = tool_executor.invoke(action)
    # 构造 FunctionMessage
    function_message = ToolMessage(response, tool_call_id=tool_call_id)
    # 返回消息列表
    return {"messages": [function_message]}


C:\Users\陈群\AppData\Local\Temp\ipykernel_13384\391848375.py:6: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


Finally, we define a function for the conditional edge, to help us figure out which direction to go (tool or user response)

We can benefit from the agent (LLM) response in LangChain, which has additional_kwargs to make a function_call with the name of the tool.

So our logic is, if function_call available in the additional_kwargs, then call tool if not then end the discussion and respond back to the user

In [166]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if "tool_calls" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

Now with all of the changes above, our LangGraph app is modified as below:

In [167]:
# from langgraph.graph import Graph, END

# workflow = Graph()

# Or you could import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")


app = workflow.compile()

We also pass the first message using HumanMessage component available in langchain, makes it easy to differentiate from AIMessage, and FunctionMessage

In [162]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas?")]}#what is the temperature in las vegas
result = app.invoke(inputs)

C:\Users\陈群\AppData\Local\Temp\ipykernel_13384\391848375.py:24: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


In [168]:
inputs = {"messages": [HumanMessage(content="how are you today?")]}#what is the temperature in las vegas
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with anything you need! How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 77, 'total_tokens': 113, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'stop', 'logprobs': None}, id='run-aaa97084-8245-4ac9-83fa-bd145e74a9f7-0', usage_metadata={'input_tokens': 77, 'output_tokens': 36, 'total_tokens': 113, 'input_token_details': {}, 'output_token_details': {}})]}

---



In [158]:
inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas?")]}#what is the temperature in las vegas
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-Buc2CfgN8ehT7rvVwdlZKjJABHVR8', 'function': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 80, 'total_tokens': 96, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fc826765-2a9f-4f15-aecc-b3f7036f1197-0', tool_calls=[{'name': 'open_weather_map', 'args': {'location': 'Las Vegas'}, 'id': 'chatcmpl-Buc2CfgN8ehT7rvVwdlZKjJABHVR8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 16, 'total_tokens': 96, 'input_token_details': {}, 'output_token_details': {}})]}

---

last_message===
 content='' additional_kwargs={'tool_calls': [{'id': 'chatcmpl-Buc2CfgN8ehT7rvVwdlZKjJA

C:\Users\陈群\AppData\Local\Temp\ipykernel_13384\1869951209.py:28: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


response===
 In Las Vegas, the current weather is as follows:
Detailed status: clear sky
Wind speed: 5.36 m/s, direction: 0°
Humidity: 35%
Temperature: 
  - Current: 13.53°C
  - High: 14.51°C
  - Low: 10.88°C
  - Feels like: 11.85°C
Rain: {}
Heat index: None
Cloud cover: 0%
function_message==
 content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 5.36 m/s, direction: 0°\nHumidity: 35%\nTemperature: \n  - Current: 13.53°C\n  - High: 14.51°C\n  - Low: 10.88°C\n  - Feels like: 11.85°C\nRain: {}\nHeat index: None\nCloud cover: 0%' tool_call_id='chatcmpl-Buc2CfgN8ehT7rvVwdlZKjJABHVR8'
Output from node 'tool':
---
{'messages': [ToolMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 5.36 m/s, direction: 0°\nHumidity: 35%\nTemperature: \n  - Current: 13.53°C\n  - High: 14.51°C\n  - Low: 10.88°C\n  - Feels like: 11.85°C\nRain: {}\nHeat index: None\nCloud cover: 0%', tool_call_id='chatcmpl-Buc2Cf

Hopefully, that gives you a good understanding of how we built a LangGraph app and why we used different LC components.

下面给出完整代码

In [169]:
import json
from langgraph.graph import Graph, END,StateGraph
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.prebuilt import ToolInvocation, ToolNode,ToolExecutor  # 引入 ToolNode

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(
            openai_api_base=OPENAI_API_BASE,
            openai_api_key=OPENAI_API_KEY,
            temperature=0)

functions = [convert_to_openai_function(t) for t in tools]

model = model.bind_tools(functions)

def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

tool_executor = ToolExecutor(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1]  # 取最后一条消息，获取要发送给工具的查询
    print('last_message===\n',last_message)

    # 确保 tool_calls 存在且为非空列表
    tool_calls = last_message.additional_kwargs.get("tool_calls", [])

    if tool_calls:
        function_data = tool_calls[0].get("function", {})  # 获取 function 字典
        arguments_str = function_data.get("arguments", "{}")  # 获取 arguments JSON 字符串
        parsed_tool_input = json.loads(arguments_str)  # 解析 JSON
        print('parsed_tool_input===\n', parsed_tool_input)
        tool_call_id=tool_calls[0]["id"]
    else:
        print("Warning: tool_calls is empty.")
    
    print('function_data===\n',function_data,function_data["name"])
    print('parsed_tool_input===\n',parsed_tool_input,parsed_tool_input['location'])
    # 构造 ToolInvocation
    action = ToolInvocation(
        tool=function_data["name"],
        tool_input=parsed_tool_input['location'],
    )

    # 使用 tool_executor 处理请求
    response = tool_executor.invoke(action)
    print('response===\n',response,'\n',action.tool)
    # 构造 FunctionMessage
    function_message = ToolMessage(response, tool_call_id=tool_call_id)

    # 返回消息列表
    return {"messages": [function_message]}

def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if "tool_calls" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"
    

# from langgraph.graph import Graph, END
# workflow = Graph()

# Or you could import StateGraph and pass AgentState to it
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")

app = workflow.compile()


inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas?")]} # what is the temperature in las vegas
result = app.invoke(inputs)
print('type result=====\n\n\n',type(result))
print('result=====\n\n\n',result)

C:\Users\陈群\AppData\Local\Temp\ipykernel_13384\3737097709.py:24: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


last_message===
 content='' additional_kwargs={'tool_calls': [{'id': 'chatcmpl-c8tdWcPD6h68XZXYEX3I1lIDLPZn6', 'function': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 80, 'total_tokens': 96, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-53715c56-b036-4721-a09e-376a950fbf5c-0' tool_calls=[{'name': 'open_weather_map', 'args': {'location': 'Las Vegas'}, 'id': 'chatcmpl-c8tdWcPD6h68XZXYEX3I1lIDLPZn6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 80, 'output_tokens': 16, 'total_tokens': 96, 'input_token_details': {}, 'output_token_details': {}}
parsed_tool_input===
 {'location': 'Las Vegas'}
function_data===
 {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'} open_weather_map
p

C:\Users\陈群\AppData\Local\Temp\ipykernel_13384\3737097709.py:46: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


response===
 In Las Vegas, the current weather is as follows:
Detailed status: clear sky
Wind speed: 5.36 m/s, direction: 0°
Humidity: 35%
Temperature: 
  - Current: 13.53°C
  - High: 14.51°C
  - Low: 10.88°C
  - Feels like: 11.85°C
Rain: {}
Heat index: None
Cloud cover: 0% 
 open_weather_map
type result=====


 <class 'langgraph.pregel.io.AddableValuesDict'>
result=====


 {'messages': [HumanMessage(content='what is the temperature in las vegas?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-c8tdWcPD6h68XZXYEX3I1lIDLPZn6', 'function': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 80, 'total_tokens': 96, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'tool_calls', 'logp

In [145]:
# 取出 messages 列表中第一个消息对象
first_msg = result["messages"][-1]

# 获取第一个消息的 content 属性
first_content = first_msg.content

# 打印第一个消息的内容
print(first_content)


The current temperature in Las Vegas is 13.53°C.
